In [ ]:
##! IGNORE THIS if running on Google Colab
%load_ext notexbook

%texify

<img src="https://colab.research.google.com/img/colab_favicon_256px.png" width="5%" class="badges" />

In [ ]:
# UNCOMMENT THIS ONLY if running on Google Colab

# !pip install syft==0.5.1
# !pip install protobuf==3.20

**ORIGINAL NOTEBOOK** [here](https://github.com/OpenMined/courses/tree/foundations-of-private-computation/federated-learning/duet_iris_classifier) from the PrivateAI Series

In [ ]:
import syft as sy

In [ ]:
sy.__version__

# Part 1: Launch a Duet Server

In [ ]:
duet = sy.launch_duet()  # option to add loopback=True if running locally

# Part 2: Upload data to Duet Server

Let's say the data owner has a dataset of Iris flowers. He will upload the data to the duet server for other data scientists to use.

In [ ]:
from sklearn import datasets
import torch as th
import numpy as np

SEED = 123456
np.random.seed(SEED)

In [ ]:
iris = datasets.load_iris()
X, y = iris.data, iris.target

Flower species mappings:
1. "Iris-setosa": 0,
2. "Iris-versicolor": 1,
3. "Iris-virginica": 2

Flower features:
1. sepal length (cm)
2. sepal width (cm)
3. petal length (cm)
4. petal width (cm)

In [ ]:
print("data:")
print(X[0:5])

In [ ]:
print("target:")
print(y)

In [ ]:
print("Length of dataset:", len(X))

In [ ]:
print(type(X))

In [ ]:
from sklearn.model_selection import train_test_split

X_train, _, y_train, _ = train_test_split(X, y, random_state=SEED, test_size=0.2)

In [ ]:
X_train.shape, y_train.shape

For doing machine learning using torch, we need the data to be converted to `FloatTensors`. 

Here, the data owner is explicitly doing the conversion before uploading the data. 

If he doesn't do that, it has to be converted in the data scientist's end.

In [ ]:
X = th.FloatTensor(X_train)
y = th.FloatTensor(y_train)

In [ ]:
print(type(X))

In [ ]:
X = X.tag("iris-data")
y = y.tag("iris-target")

X = X.describe(
    "This is a dataset for flower classification of 150 samples. 4 Features are sepal length (cm),"
    "sepal width (cm), petal length (cm), petal width (cm)"
)
y = y.describe("Labels for flowers: Iris-setosa, Iris-versicolour, Iris-virginica")

In [ ]:
data_pointer = X.send(duet, pointable=True)
target_pointer = y.send(duet, pointable=True)

In [ ]:
# Once uploaded, the data owner can see the object stored in the tensor
duet.store

In [ ]:
# To see it in a human-readable format, data owner can also pretty-print the tensor information
duet.store.pandas

# Part 3: Response to requests coming from Data Scientist

The data owner can add requests to be accepted or denied by adding them to request handlers. If he doesn't specify a `name`, then all the requests will be accepted.

In [ ]:
duet.requests.add_handler(action="accept")

### <img src="https://github.com/OpenMined/design-assets/raw/master/logos/OM/mark-primary-light.png" alt="he-black-box" width="100"/> Checkpoint 1 : Well done!